# Module 2 Assignment

Using the approach demonstrated up to page #71 of the slide and that of provided in the previous modules and demonstrate that:

1.You can explore materials available in the internet and used them to explain how to make the example on pg #2 of this module run using multicores or multithreads

2.You can transform all the matrixes of the example on pg #2 of this module to be in csr format and modify the code to let the csr-implemented yield the same result as its original. 

3.You can transform all the matrixes of the example on pg #2 of this module to be in coo format and modify the code to let the coo-implemented yield the same result as its original. Then compare the performance with (2)

# preparing the essential library

In [1]:
import pandas as pd # do some data
import string
import timeit # just import for timer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Code from page 71

### the retrive data part

In [2]:
def get_and_clean_data():
    data = pd.read_csv('./data/software_developer_united_states_1971_20191023_1.csv')
    description = data['job_description']
    cleaned_description = description.apply(lambda s: s.translate(str.maketrans('', '', string.punctuation + u'\xa0')))
    cleaned_description = cleaned_description.apply(lambda s: s.lower())
    cleaned_description = cleaned_description.apply(lambda s: s.translate(str.maketrans(string.whitespace, ' '*len(string.whitespace), '')))
    cleaned_description = cleaned_description.drop_duplicates()
    return cleaned_description

### The data cleaning part

In [3]:
def clean_des() :
    cleaned_description = get_and_clean_data()
    cleaned_description = cleaned_description.iloc[:2]
    
    tokenized_description = cleaned_description.apply(lambda s : word_tokenize(s))
    
    sw_removed_description = tokenized_description.apply(
        lambda s : [
            word for word in s if word not in stopwords.words()
        ]
    )
    
    sw_removed_description = sw_removed_description.apply(
        lambda s: [
            word for word in s if len(word) > 2
        ]
    )
    
    ps = PorterStemmer()
    
    stemmed_description = sw_removed_description.apply(
        lambda s: [
            ps.stem(w) for w in s
        ]
    )
    
    return stemmed_description

### some example using those two function

In [12]:
stemmed_description = clean_des()

cv = CountVectorizer(analyzer=lambda x:x)
X = cv.fit_transform(stemmed_description)

print(pd.DataFrame(X.toarray()))

   0    1    2    3    4    5    6    7    8    9    ...  277  278  279  280  \
0    0    1    1    1    1    0    1    1    1    0  ...    1    1    3    1   
1    1    0    0    0    0    1    0    0    0    1  ...    0    0    1    0   

   281  282  283  284  285  286  
0    2    1    0    5    1    0  
1    1    0    1    1    0    1  

[2 rows x 287 columns]


### another example from those two function above

In [10]:
timeit.timeit(lambda: np.matmul(X.toarray(),X.toarray().T),number=1)
np.shape(np.matmul(X.toarray(),X.toarray().T))

timeit.timeit(lambda: X*X.T,number=1)
np.shape(X*X.T)

(2, 2)

---

# This is code from page 2

### clean data part

In [4]:
def simple_tokenize(data) :
    cleaned_description = data.apply(lambda s: [x.strip() for x in s.split()])
    return cleaned_description

In [5]:
def parse_job_description():
    cleaned_description = get_and_clean_data()
    cleaned_description = simple_tokenize(cleaned_description)
    return cleaned_description

### The code

In [18]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [9]:
from tabulate import tabulate

parsed_description = parse_job_description()

sw_set = set(stopwords.words()) - {'c'}
no_sw_description = parsed_description.apply(
    lambda x: [w for w in x if w not in sw_set]
)

ps = PorterStemmer()

stemmed_description = no_sw_description.apply(
    lambda x: set([ps.stem(w) for w in x])
)

all_unique_term = list(
    set.union(*stemmed_description.to_list())
)

invert_idx = {}

for s in all_unique_term :
    invert_idx[s] = set(
        stemmed_description.loc[
            stemmed_description.apply(
                lambda x : s in x
            )
        ].index
    )


# search mongodb
to_search_db = ['mongodb']

stemmed_db = np.unique([
    ps.stem(w) for w in to_search_db
])

searched_db = set.union(
    *[invert_idx[s] for s in stemmed_db]
)

# search java
to_search_lang = ['java']

stemmed_lang = np.unique([
    ps.stem(w) for w in to_search_lang
])

searched_lang = set.union(
    *[invert_idx[s] for s in stemmed_lang]
)

appear_both = searched_db.intersection(searched_lang)

print(
    parsed_description.loc[appear_both].apply(
        lambda x: ' '.join(x)
    ).tail().to_markdown()
)

|      | job_description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

---

# The Assignment part

#### 1.You can explore materials available in the internet and used them to explain how to make the example on pg #2 of this module run using multicores or multithreads

In [6]:
import multiprocessing as mp
print("My computer can process",mp.cpu_count(),"jobs in parallel")

My computer can process 8 jobs in parallel


#### Preparing the multiprocessing computing

In [ ]:
parsed_description_modified = parse_job_description()

sw_set_modified = set(stopwords.words()) - {'c'}
no_sw_description_modified = parsed_description_modified.apply(
    lambda x: [w for w in x if w not in sw_set_modified]
)   

ps_modified = PorterStemmer()

stemmed_description_modified = no_sw_description_modified.apply(
    lambda x: set([ps_modified.stem(w) for w in x])
)

all_unique_term_modified = list(
    set.union(*stemmed_description_modified.to_list())
)

invert_idx_modified = {}

for s in all_unique_term_modified :
    invert_idx_modified[s] = set(
        stemmed_description_modified.loc[
            stemmed_description_modified.apply(
                lambda x : s in x
            )
        ].index
    )

#### seperate the code that can do a pararell

In [16]:
def searching(keyword) :
    to_search = [keyword]

    stemmed = np.unique([
        ps_modified.stem(w) for w in to_search
    ])

    searched = set.union(
        *[invert_idx_modified[s] for s in stemmed]
    )
    return searched

In [17]:
def keyword_intersac(key1,key2) :
    return key1.intersection(key2)

#### using Multiprocessing pool to do a multiprocessing stuff

In [18]:
def core_search() :
    pool = mp.Pool(processes=8)
    lang_mod = pool.apply_async(searching,('java'))
    db_mod = pool.apply_async(searching,('mongodb'))
    appear_both_modified = pool.apply_async(keyword_intersac,(db_mod.get(),lang_mod.get()))
    return appear_both_modified.get()

In [ ]:
both_mode = core_search()
print(
    parsed_description_modified.loc[both_mode].apply(
        lambda x: ' '.join(x)
    ).tail().to_markdown()
)

#### 2.You can transform all the matrixes of the example on pg #2 of this module to be in csr format and modify the code to let the csr-implemented yield the same result as its original.

In [15]:
cv = CountVectorizer(analyzer=lambda x:x)
X = cv.fit_transform(
    parsed_description.loc[appear_both].apply(
        lambda x: ' '.join(x)
    )
)

# original timing
timeit.timeit(lambda: np.matmul(X.toarray(),X.toarray().T),number=1)
np.shape(np.matmul(X.toarray(),X.toarray().T))

print("original result")
print(X[0, :])
timeit.timeit(lambda: X*X.T,number=1)

original result
  (0, 28)	476
  (0, 15)	864
  (0, 11)	455
  (0, 14)	249
  (0, 0)	1131
  (0, 33)	90
  (0, 18)	126
  (0, 30)	542
  (0, 26)	294
  (0, 25)	521
  (0, 22)	387
  (0, 29)	483
  (0, 35)	85
  (0, 19)	652
  (0, 24)	534
  (0, 17)	214
  (0, 12)	97
  (0, 31)	211
  (0, 21)	47
  (0, 32)	106
  (0, 20)	28
  (0, 13)	306
  (0, 16)	132
  (0, 23)	156
  (0, 27)	42
  (0, 36)	2
  (0, 2)	2
  (0, 34)	38
  (0, 46)	2
  (0, 49)	10
  (0, 8)	2
  (0, 6)	4
  (0, 4)	6
  (0, 53)	2
  (0, 3)	2


0.002322899999967376

In [16]:
# csr timing
print(X.tocsr())
print("totoal time")
timeit.timeit(lambda: X.tocsr()*X.T.tocsr(),number=1)

  (0, 28)	476
  (0, 15)	864
  (0, 11)	455
  (0, 14)	249
  (0, 0)	1131
  (0, 33)	90
  (0, 18)	126
  (0, 30)	542
  (0, 26)	294
  (0, 25)	521
  (0, 22)	387
  (0, 29)	483
  (0, 35)	85
  (0, 19)	652
  (0, 24)	534
  (0, 17)	214
  (0, 12)	97
  (0, 31)	211
  (0, 21)	47
  (0, 32)	106
  (0, 20)	28
  (0, 13)	306
  (0, 16)	132
  (0, 23)	156
  (0, 27)	42
  :	:
  (165, 18)	176
  (165, 30)	574
  (165, 26)	194
  (165, 25)	525
  (165, 22)	293
  (165, 29)	393
  (165, 35)	107
  (165, 19)	564
  (165, 24)	626
  (165, 17)	214
  (165, 12)	111
  (165, 31)	177
  (165, 21)	53
  (165, 32)	106
  (165, 20)	10
  (165, 13)	266
  (165, 16)	110
  (165, 23)	172
  (165, 27)	8
  (165, 36)	18
  (165, 2)	2
  (165, 34)	50
  (165, 46)	8
  (165, 49)	12
  (165, 1)	4
totoal time


0.01029900000003181

#### 3.You can transform all the matrixes of the example on pg #2 of this module to be in coo format and modify the code to let the coo-implemented yield the same result as its original. Then compare the performance with (2)

In [17]:
# coo timing
print(X.tocoo())
print("total time")
timeit.timeit(lambda: X.tocoo()*X.T.tocoo(),number=1)

  (0, 28)	476
  (0, 15)	864
  (0, 11)	455
  (0, 14)	249
  (0, 0)	1131
  (0, 33)	90
  (0, 18)	126
  (0, 30)	542
  (0, 26)	294
  (0, 25)	521
  (0, 22)	387
  (0, 29)	483
  (0, 35)	85
  (0, 19)	652
  (0, 24)	534
  (0, 17)	214
  (0, 12)	97
  (0, 31)	211
  (0, 21)	47
  (0, 32)	106
  (0, 20)	28
  (0, 13)	306
  (0, 16)	132
  (0, 23)	156
  (0, 27)	42
  :	:
  (165, 18)	176
  (165, 30)	574
  (165, 26)	194
  (165, 25)	525
  (165, 22)	293
  (165, 29)	393
  (165, 35)	107
  (165, 19)	564
  (165, 24)	626
  (165, 17)	214
  (165, 12)	111
  (165, 31)	177
  (165, 21)	53
  (165, 32)	106
  (165, 20)	10
  (165, 13)	266
  (165, 16)	110
  (165, 23)	172
  (165, 27)	8
  (165, 36)	18
  (165, 2)	2
  (165, 34)	50
  (165, 46)	8
  (165, 49)	12
  (165, 1)	4
total time


0.012671500000010383